# BlueBerry Winery - Wine Quality Prediction with Machine Learning

Develop models to determine the quality of the wines produced based on their chemical composition

## Load libraries and data

In [107]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


#do i need to import submodules like minmaxscaler separately?


In [108]:
red_wine = pd.read_csv('winequality-red.csv', sep=';')
white_wine = pd.read_csv('winequality-white.csv', sep=';')
wines = pd.read_csv('wines_combined.csv')
display(wines)

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,quality_label,color
0,0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,low,red
1,1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,low,red
2,2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,low,red
3,3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,medium,red
4,4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,low,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,medium,white
6493,4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,low,white
6494,4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,medium,white
6495,4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,medium,white


## Data Wrangling 2

### 0. Do I need to declare the index row?

### 1. Change column headings to snake_case and use IUPAC standard spelling ('sulfates')

In [109]:
display(wines)
wines = wines.rename(columns={"fixed acidity": "fixed_acidity", "volatile acidity": "volatile_acidity", "citric acid": "citric_acid", "residual sugar" : "residual_sugar", "free sulfur dioxide" : "free_sulfur_dioxide", "total sulfur dioxide" : "total_sulfur_dioxide", "sulphates" : "sulfates"})
display(wines)

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,quality_label,color
0,0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,low,red
1,1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,low,red
2,2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,low,red
3,3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,medium,red
4,4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,low,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,medium,white
6493,4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,low,white
6494,4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,medium,white
6495,4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,medium,white


,Unnamed: 0,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulfates,alcohol,quality,quality_label,color
0,0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,low,red
1,1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,low,red
2,2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,low,red
3,3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,medium,red
4,4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,low,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,medium,white
6493,4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,low,white
6494,4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,medium,white
6495,4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,medium,white


### 2. Drop old index values form red/white datasets

In [110]:
wines = wines.drop(['Unnamed: 0'], axis=1)

In [111]:
display(wines)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulfates,alcohol,quality,quality_label,color
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,low,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,low,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,low,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,medium,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,low,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,medium,white
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,low,white
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,medium,white
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,medium,white


### 3. Drop duplicates

In [112]:
# First, check for duplicates using duplicated() ## FIXED now that second index removed
# use the value counts of duplicated() to show how many items are duplicated
display('Counts of duplicated items:', wines.duplicated().value_counts())

## workaround, no longer needed #############
# Make a a new df contining only numerical data and no index (indexes are always unique and might cause a false negative)
#df_dup_compare = wines.loc[:,'fixed acidity':'quality']

# use the value counts of duplicated() to show how many items are duplicated
#display("Counts of items that are duplications of previous items (in new df):", df_dup_compare.duplicated().value_counts())

# display all the rows of the duplicated() output series where duplicated==True
## use keep=False to (counterintuitively) keep all instances of duplicated entries
#display("All indices with dupicated data:", df_dup_compare.duplicated(keep=False)[lambda x : x == True])

# display all the rows of the df where duplicated == True
#wines_duplicates = df_dup_compare[df_dup_compare.duplicated() == True]
#display("All df rows which are duplications:", wines_duplicates)

# write csv of duplicated data for checking
#wines_duplicates.to_csv('wines_duplicates.csv')
#####################################

wines_clean = wines.drop_duplicates(keep='last')
display(wines_clean)

'Counts of duplicated items:'

False    5320
True     1177
dtype: int64

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulfates,alcohol,quality,quality_label,color
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,low,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,low,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,medium,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,low,red
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5,low,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,medium,white
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,low,white
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,medium,white
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,medium,white


### 4. Double-check data types

In [113]:
wines_clean.dtypes

fixed_acidity           float64
volatile_acidity        float64
citric_acid             float64
residual_sugar          float64
chlorides               float64
free_sulfur_dioxide     float64
total_sulfur_dioxide    float64
density                 float64
pH                      float64
sulfates                float64
alcohol                 float64
quality                   int64
quality_label            object
color                    object
dtype: object

In [114]:
# cast columns to Categorical (pandas) data type:
# color, unordered
wines_clean['color'] = pd.Categorical(wines_clean['color'],
categories=['red', 'white'], ordered=False)

# quality_label, ordered low to high
wines_clean['quality_label'] = pd.Categorical(wines_clean['quality_label'],
categories=['low', 'medium', 'high'], ordered=True)

wines_clean.dtypes

display(wines_clean)

/tmp/ipykernel_40167/410774185.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wines_clean['color'] = pd.Categorical(wines_clean['color'],
/tmp/ipykernel_40167/410774185.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wines_clean['quality_label'] = pd.Categorical(wines_clean['quality_label'],


,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulfates,alcohol,quality,quality_label,color
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,low,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,low,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,medium,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,low,red
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5,low,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,medium,white
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,low,white
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,medium,white
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,medium,white


### 5. Calculate skewness and kurtosis

Skewness: outside -0.5 and +0.5 is considered highly skewed [source?]

In [115]:
print((wines_clean.select_dtypes(include=['int64', 'float64'])).astype(float).skew())

fixed_acidity           1.650417
volatile_acidity        1.504557
citric_acid             0.484309
residual_sugar          1.706550
chlorides               5.338237
free_sulfur_dioxide     1.362719
total_sulfur_dioxide    0.063614
density                 0.666326
pH                      0.389969
sulfates                1.809454
alcohol                 0.545696
quality                 0.147467
dtype: float64


Kurtosis: XXX is considered high [source]

In [116]:
print((wines_clean.select_dtypes(include=['int64', 'float64'])).astype(float).kurt())

fixed_acidity            4.589079
volatile_acidity         2.863175
citric_acid              2.582471
residual_sugar           7.025595
chlorides               48.260708
free_sulfur_dioxide      9.520706
total_sulfur_dioxide    -0.299997
density                  8.711498
pH                       0.431811
sulfates                 8.612917
alcohol                 -0.538169
quality                  0.298100
dtype: float64


## Data preprocessing for ML 1

### 1. Encoding categorical variables

Use scikit. Resource: https://scikit-learn.org/stable/modules/preprocessing.html 

Resource: https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd

Resource: https://scikit-learn.org/stable/modules/preprocessing_targets.html#preprocessing-targets

#### Quality Label (Ordinal: Label Encoder)

In [117]:
# add a new variable storing a numerical code for each category
## note: didn't use LabelEncoder, used .cat.codes as per https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd

wines_clean['quality_label_cat'] = wines_clean['quality_label'].cat.codes 
display(wines_clean)

/tmp/ipykernel_40167/1497370397.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wines_clean['quality_label_cat'] = wines_clean['quality_label'].cat.codes


,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulfates,alcohol,quality,quality_label,color,quality_label_cat
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,low,red,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,low,red,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,medium,red,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,low,red,0
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5,low,red,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,medium,white,1
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,low,white,0
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,medium,white,1
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,medium,white,1


In [118]:
display(wines_clean.quality_label_cat.value_counts(), wines_clean.quality_label.value_counts())

1    3179
0    1988
2     153
Name: quality_label_cat, dtype: int64

medium    3179
low       1988
high       153
Name: quality_label, dtype: int64

#### Color (Nominal: One-Hot Encoder)

In [119]:
# OHE creates new dummy variable columns for each category, with binary encoding (one of them 1, all others 0) indicating whether the category is applied

# Step 1: OHE needs numerica data, so first transform categorical data using LabelEncoder() [This time really use LabelEncoder!]

# create an instance of labelencoder  -TODO: explain this more
labelencoder = LabelEncoder()
# make new column and apply numerical category values
wines_clean['color_cat'] = labelencoder.fit_transform(wines_clean['color'])

# Step 2: Now use OneHotEncoder. First, create an instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')

# pass in color_cat to make a df containing the multiple (here, 2) binary encoding columns
enc_df = pd.DataFrame(enc.fit_transform(wines_clean[['color_cat']]).toarray())
display('encoder array:', enc_df)
# merge df with wines_clean
wines_clean = wines_clean.join(enc_df)

display(wines_clean)

/tmp/ipykernel_40167/3432885863.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wines_clean['color_cat'] = labelencoder.fit_transform(wines_clean['color'])


'encoder array:'

,0,1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
...,...,...
5315,0.0,1.0
5316,0.0,1.0
5317,0.0,1.0
5318,0.0,1.0


,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulfates,alcohol,quality,quality_label,color,quality_label_cat,color_cat,0,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,low,red,0,0,1.0,0.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,low,red,0,0,1.0,0.0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,medium,red,1,0,1.0,0.0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,low,red,0,0,1.0,0.0
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5,low,red,0,0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,medium,white,1,1,NaN,NaN
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,low,white,0,1,NaN,NaN
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,medium,white,1,1,NaN,NaN
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,medium,white,1,1,NaN,NaN


In [127]:
## TODO:
## ERROR: zeroes are joined to main df as NaN. also they are floats?? could be related, but the example also had floats 
##at https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd
## also, there was no step to retain column labels... look here under 'names' https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

wines_clean.dtypes

fixed_acidity            float64
volatile_acidity         float64
citric_acid              float64
residual_sugar           float64
chlorides                float64
free_sulfur_dioxide      float64
total_sulfur_dioxide     float64
density                  float64
pH                       float64
sulfates                 float64
alcohol                  float64
quality                    int64
quality_label           category
color                   category
quality_label_cat           int8
color_cat                  int64
0                        float64
1                        float64
dtype: object

### 2. Split data

Use`test_train_split` to separate testing and traning portions of the data. [criteria for % proportion, source]

Resource: https://machinelearningmastery.com/difference-test-validation-datasets/

In [120]:
# X = df # --> the features we will keep to build our model
# y = target # --> what you're trying to predict

# # Example:
# y=X.SalePrice
# X.drop(['SalePrice'],axis=1,inplace=True)
# print(y)

# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

## Apply ML Model: Logistic Regression

### 1. Fit the model

In [121]:
# # Example
# # Fit the model
# lr = LogisticRegression()
# lr.fit(X_train, y_train)

### 2. Make predicitons

In [122]:
# #Now you will apply the .predict() method to make predictions on test data. These predictions are stored in 'pred_lr'.

# # Make predictions
# pred_lr = lr.predict(X_test)

## Evaluate ML Model (Logistic Regression)

### 1. Confusion Matrix

The confusion matrix function returns a table of true/false - negative/positive predictions made by the model. It is not an evaluaiton metric but goves an overview.

In [123]:
# print("Confusion matrix:")
# print(confusion_matrix(y_test, pred_lr))

### 2. Accuracy Score (not useful with imbalanced classes)

### 3. Classification Report

In [124]:
#print(classification_report(y_test, pred_lr, target_names=["low","medium"]))


### 4. Cohen-Kappa Score

A measure of classifier performance compared to a random-guessing model, especially good for an imbalanced data set.


Resources:
- https://analyticsindiamag.com/understanding-cohens-kappa-score-with-hands-on-implementation/
- https://www.knime.com/blog/cohens-kappa-an-overview



## Data preprocessing for ML 2

### 1. Feature reduction (round 1)

Goal: drop highly correlated features and drop features that do not help in the prediction.

https://machinelearningmastery.com/calculate-feature-importance-with-python/

### 2. Perform Feature scaling

#### Normalisation

In [125]:
#MinMaxScaler

#### Standardisation

In [126]:
# StandardScaler: It shrinks the range of the feature values. However, this scaler doesn't work correctly in the presence of outliers.
# RobustScaler: If your data set contains outliers, this scaler is more appropriate because it works with the 1st and 3rd quantiles and not the mean and variance scaling.

# data normalisation with sklearn
# from sklearn.preprocessing import MinMaxScaler

# # fit scaler on training data
# norm = MinMaxScaler().fit(X_train)

# # transform training data
# X_train_norm = norm.transform(X_train)

# # transform testing data
# X_test_norm = norm.transform(X_test)

### 3. Remove outliers??

Resource: https://www.pluralsight.com/guides/cleaning-up-data-from-outliers

## Improve ML Model (Logistic Regression)

### 2. Parameter tuning and model improvement with k-Fold cross-validation

use the k-Fold Validation to perform a validation of the training set for hyper-parameter tuning 
 
(check the Machine Learning Fundamentals chapter on Cross-Validation for more details). https://lms.codeacademyberlin.com/content/data/Machine-Learning-Fundamentals#:~:text=is%20called%20validation.-,Validation%20and%20K%2Dfold%20Cross%2DValidation,-Validation%20is%20a

For this task, you can use the scikit-learn method cross_val_score (NB- not the same as cross_validate)

## Apply ML Model: XXXXXXXX 

repeat all steps for new model